In [50]:
import xlrd
from datetime import datetime
from dateutil import parser
from string import Template
import os

sFile_name = "1st April "

In [51]:
workbook = xlrd.open_workbook('data/order_data_completed/1ST JAN 2019.xlsx')

In [52]:
sheet = workbook.sheet_by_index(0)

In [53]:
sheet.cell_value(337,4)

10.0

In [55]:
from string import Template 

In [56]:
int(Template(['$i',0][0]).substitute({'i':1}))

1

In [57]:
try:
    print(sheet.cell_value(4,26))
except IndexError as err:
    print("There is a index error: {}".format(err))

PayMode


In [58]:
# list_cols = {sheet.cell_value(4,i).replace(' ','_').replace('.','_').replace('%','').replace('#','').replace('&','and') : i  for i in range(19)} 
# # list_cols = [{"Restaurant":-1}] + list_cols
# list_cols["UserID"] = 26
# list_cols["PayMode"] = 24
# list_cols["Tips"] = 25
# # list_cols["Restaurant"] = -1
# list_cols


In [59]:
# list_cols.get('Tips')

(0,0) : heading of excel
(1,0) : Date
(2,0) : Restaurant name
(3,0) : Empty
(4,0) : S.No


Start from here j = 0, i = 4


(i+1,0) : Restaurant Category  #start_data_reading
(i+2,0) : Empty ----  ( Begining of data is here )

-- Module to read data . #read_data (i)
(i+3,0) : Data Starts from here j = i loop starts here while

        j+=1

(j,0) : is empty 
        (j+1,8) : Classification
                If Not epmty continue reading data. #read_data i = j+1
                If Empty check 
                       IF (i+1, 0)
                               is Total Bill #start_data_reading
                               
                      
        





In [60]:
sheet.nrows

994

In [61]:

# oDict = {i:"" for i in list_cols.keys()}
# oDict['Restaurant'] = ""
# print(oDict)

# sTemplate = "$"+",$".join(oDict.keys())
# print(sTemplate)
# t = Template(sTemplate)

In [62]:
def get_template(list_cols):
    oDict = {i:"" for i in list_cols.keys()}
    oDict['Restaurant'] = ""

    sTemplate = "$"+",$".join(oDict.keys())
    t = Template(sTemplate)
    return t

def get_original_cols(sheet):
    c = {}
    h = "@@"
    i = 0
    while h and i < sheet.ncols:
        h = sheet.cell_value(4,i).replace(' ','_').replace('.','_').replace('%','').replace('#','').replace('&','and')
        c[h] = i
        i+=1
    return c

def get_cols():
    return {'S_No': 0,
             'BillDate_and_Time': 0,
             'BillNo': 0,
             'Tbl_': 0,
             'Pax': 0,
             'KotNo': 0,
             'KOTDate_and_Time': 0,
             'Steward': 0,
             'Classification': 0,
             'MenuGroup': 0,
             'ItemCode': 0,
             'ItemName': 0,
             'Rate': 0,
             'Qty': 0,
             'Amount': 0,
             'Discount': 0,
             'CGS_2_5': 0,
             'SGS_2_5': 0,
             'Total': 0,
             'UserID': 0,
             'PayMode': 0,
             'Tips': 0}



In [63]:

# Variables
sRestaurant = ""
iSerialNo = 0
billdate_and_time = ""
iBillNo=0
sTable = ""
iPax = 0
iKotNo = 0
sKot_date_and_time = ""
sSteward = ""
sUserdId = ""

res_name_begin_data_row = 5

# column indexes
classification_col = 8
last_col = 18
payment_model_col = 24
tips_col = 25
user_id_col = 26

# flags 
b_read_first_var = False
b_excel_ends = False

# Indices
read_data_index = 0
read_cursor = 0

# list of fixed columns
list_fixed_cols = ["S_No","BillDate_and_Time","BillNo","Tbl_", "Pax", "KotNo", "KOTDate_and_Time", "Steward"]
list_var_cols = ["Classification","MenuGroup","ItemCode","ItemName","Rate","Qty","Amount","Discount","CGS_2_5","SGS_2_5", "Total","PayMode","Tips"]
userid_col = "UserID"

date_time_cols = ["BillDate_and_Time", "KOTDate_and_Time"]

# fixed columns
oFixedDict = {}
aDataList = []

def get_date_format(date_string):
    if '/' in date_string:
        return '%d/%m/%Y  %H.%M'
    else:
        return '%d-%b-%Y  %H.%M'

    


def extractData(sheet, sRestaurant, read_data_index, aData, list_cols):
    classification_col = list_cols["Classification"]        
    bFlag = False
    if sheet.cell_value(4,2).strip() == 'Time':
        bFlag = True
    for x in list_fixed_cols:
        g = sheet.cell_value(read_data_index,list_cols[x])
        if x in date_time_cols:
            try:
                if bFlag and x == "BillDate_and_Time":
                    g = g + "  " + sheet.cell_value(read_data_index,list_cols[x]+1)
                    oFixedDict[x] = datetime.strptime(g, get_date_format(g)).isoformat()
                else:
                    oFixedDict[x] = datetime.strptime(g, get_date_format(g)).isoformat()
            except Exception as e:
                oFixedDict[x] = '2030-12-31T00:00:00'
        else:
            oFixedDict[x] = g if type(g) == float else g.strip().replace(',','')
    
    while True:
        if not sheet.cell_value(read_data_index,classification_col):
            sUserdId = sheet.cell_value((read_data_index-1),list_cols["UserID"]).strip()
            
            for x in range(len(aData)):
                aData[x]["UserID"] = sUserdId
            break;
        oDict = {i:"" for i in list_cols.keys()}
        oDict['Restaurant'] = sRestaurant
        
        for key, val in list_cols.items():
            g = sheet.cell_value(read_data_index,val)
            oDict[key] = g if type(g) == float else g.strip().replace(',','')
        
        oDict.update(oFixedDict)
        aData.append(oDict)
        read_data_index += 1
    return aData, read_data_index

In [64]:

def prepareCsvData(sheet, filename):
    read_data_index = 5
    aDataList = []
    
    m = get_original_cols(sheet)
    list_cols = get_cols()
    for k,v in m.items():
        if list_cols.get(k) == 0:
            list_cols[k] = v
    
    list_cols["BillDate_and_Time"] = 1
    
    classification_col = list_cols["Classification"]
    
    t = get_template(list_cols)

    while read_data_index < sheet.nrows:
        if not sheet.cell_value(read_data_index,classification_col):
            if "restaurant" in sheet.cell_value(read_data_index,0).lower():
                sRestaurant = sheet.cell_value(read_data_index,0).split(":")[1].strip()
            read_data_index += 1
            continue

        try:
            if sheet.cell_value(read_data_index,classification_col):
                aDataList, read_data_index = extractData(sheet, sRestaurant, read_data_index, aDataList, list_cols)
        except IndexError as e:
            print(f"File is completed at {read_data_index} and classification col {classification_col} sheet rows : {sheet.nrows}")

        


    csv_list = [t.substitute(x) for x in aDataList]
    print("Completed file {} with data written >>> {}".format(filename, len(csv_list)))
    with open('./clean_data/{}.csv'.format('full_csv'), mode='a+') as f:
        for x in csv_list:
            f.write(x)
            f.write('\n')
    return read_data_index

In [65]:
list_cols = get_cols()

In [66]:
# import traceback
# if os.path.exists("./clean_data/full_csv.csv"):
#     os.remove("./clean_data/full_csv.csv")
#     print("full_csv Removed!")
    
# with open('./clean_data/{}.csv'.format('full_csv'), mode='w+') as f:
#     f.write(','.join(list_cols.keys()) + ',Restaurant')
#     f.write('\n')
    
# try:
#     x = './data/order_data_completed/1ST JAN 2019.xlsx'
#     workbook = xlrd.open_workbook(x)
#     sheet = workbook.sheet_by_index(0)
#     print(f"-----WORKING ON {x}------")
#     prepareCsvData(sheet, x)
# except Exception as e:
# #     print('file {} has following issues : \n {}'.format(x,traceback.format_exc()))
#     print('file {} has following issues : \n {}'.format(x,e))

In [67]:
'/' in '23/01/2019  14.26'

True

In [68]:
datetime.strptime('31/12/2019  00.00', '%d/%m/%Y  %H.%M').isoformat()

'2019-12-31T00:00:00'

In [69]:
g = sheet.cell_value(337,4)
g if type(g) == float else g.strip().replace(',','')
str(g)

'10.0'

In [70]:
import os
import glob2

In [71]:
list_of_files = glob2.glob("./data/order_data/*.xlsx")

In [72]:
# list_of_files = [os.path.join('april2019hatchbillingdata',x) for x in os.listdir('april2019hatchbillingdata')]

In [73]:
import traceback
if os.path.exists("./clean_data/full_csv.csv"):
    os.remove("./clean_data/full_csv.csv")
    print("full_csv Removed!")
    
with open('./clean_data/{}.csv'.format('full_csv'), mode='w+') as f:
    f.write(','.join(list_cols.keys()) + ',Restaurant')
    f.write('\n')
    
for x in list_of_files:
    try:
        workbook = xlrd.open_workbook(x)
        sheet = workbook.sheet_by_index(0)
        print(f"-----WORKING ON {x}------")
        prepareCsvData(sheet, x)
    except Exception as e:
#         print('file {} has following issues : \n {}'.format(x,traceback.format_exc()))
        print('file {} has following issues : \n {}'.format(x,e))

-----WORKING ON ./data/order_data/23RD JAN 2019.xlsx------
Completed file ./data/order_data/23RD JAN 2019.xlsx with data written >>> 440
-----WORKING ON ./data/order_data/7th JUNE 2019.xlsx------
Completed file ./data/order_data/7th JUNE 2019.xlsx with data written >>> 643
-----WORKING ON ./data/order_data/13th JUNE 2019.xlsx------
Completed file ./data/order_data/13th JUNE 2019.xlsx with data written >>> 542
-----WORKING ON ./data/order_data/1st MAY 2019 .xlsx------
Completed file ./data/order_data/1st MAY 2019 .xlsx with data written >>> 860
-----WORKING ON ./data/order_data/2nd April.xlsx------
Completed file ./data/order_data/2nd April.xlsx with data written >>> 509
-----WORKING ON ./data/order_data/8  FEB 2019.xlsx------
Completed file ./data/order_data/8  FEB 2019.xlsx with data written >>> 667
-----WORKING ON ./data/order_data/3 MARCH 2019.xlsx------
Completed file ./data/order_data/3 MARCH 2019.xlsx with data written >>> 703
-----WORKING ON ./data/order_data/13 MARCH 2019.xlsx-

In [74]:
list_cols = {'Restaurant': -2,
 'Tbl_': 0,
 'Pax': 1,
 'BillNo': 2,
 'BillDate_and_Time': 3,
 'KotOrderNo': 4,
 'KOTOrderDateAndTime': 5,
 'Steward': 6,
 'ItemName': 7,
 'Qty': 8,
 'PayMode': 10,
 'SettlementDateTime': 11,
 'UserId':14}

fixed_cols = ['Pax','BillNo','BillDate_and_Time','PayMode','SettlementDateTime','UserId']

In [75]:
# list of fixed columns
list_fixed_cols = ['Pax','BillNo','BillDate_and_Time','PayMode','SettlementDateTime','UserId']
list_var_cols = ['Tbl_',
                 'KotOrderNo',
                 'KOTOrderDateAndTime',
                 'Steward',
                 'ItemName',
                 'Qty']

# fixed columns
oFixedDict = {}
aDataList = []
date_time_cols = ["BillDate_and_Time","SettlementDateTime"]

# datetime.strptime(sheet.cell_value(7,1), '%d-%b-%Y  %H.%M')

classification_col = 5



def extractData(sheet, sRestaurant, read_data_index, aData):
    for x in list_fixed_cols:
        g = sheet.cell_value(read_data_index,list_cols[x])
        if x in date_time_cols:
            try:
                oFixedDict[x] = datetime.strptime(g, '%d-%b-%Y  %H.%M').isoformat()
            except Exception as e:
                oFixedDict[x] = '2030-12-31T00:00:00'
        else:
            oFixedDict[x] = g if type(g) == float else g.strip()

    
    while True:
        if not sheet.cell_value(read_data_index,classification_col):
            break;
        oDict = {i:"" for i in list_var_cols}
        oDict['Restaurant'] = sRestaurant
        
        for val in list_var_cols:
            g = sheet.cell_value(read_data_index,list_cols[val])
            if val == "KOTOrderDateAndTime":
               oDict[val] = datetime.strptime(g, '%d-%b-%Y  %H.%M').isoformat()
            else:
                oDict[val] = g if type(g) == float else g.strip().replace(',','')
            if g == '219':
                print('something wrong with 2190 {}'.format(oDict[val]))
        
        oDict.update(oFixedDict)
        aData.append(oDict)
        read_data_index += 1
    return aData, read_data_index

In [76]:
def prepareCsvData(sheet, filename):
    read_data_index = 6
    kill_counter = 0
    aDataList = []
    classification_col = 5
    sRestaurant = sheet.cell_value(2,7).split(':')[1].strip()
    sExcel_end_Criteria = 'total'
    t = Template("$"+",$".join(list_cols.keys()))

    while read_data_index < sheet.nrows:
        if not sheet.cell_value(read_data_index,classification_col):
            read_data_index += 1
        try:
            if sheet.cell_value(read_data_index,classification_col):
                aDataList, read_data_index = extractData(sheet, sRestaurant, read_data_index, aDataList)
        except IndexError as e:
            print(f"File is completed at {read_data_index} and classification col {classification_col} sheet rows : {sheet.nrows}")


    csv_list = [t.substitute(x) for x in aDataList]
    print("Completed file {} with data written >>> {} after cursor at {}".format(filename, len(csv_list),read_data_index))
    with open('./clean_data/{}.csv'.format('full_csv_kot'), mode='a+') as f:
        for x in csv_list:
            f.write(x)
            f.write('\n')

In [77]:
list_of_files = glob2.glob("./data/kot_data/*.xlsx")

In [78]:
# list_of_files = ['./data/kot_data/April 2019 Time spent kot Traking TIMBER .xlsx']
list_of_files

['./data/kot_data/MARCH 2019 ITEM SPENT KOT TRAKING TIMBER .xlsx',
 './data/kot_data/April 2019 Time spent kot Traking TIMBER .xlsx',
 './data/kot_data/MARCH 2019 ITEM SPENT KOT TRAKING HD.xlsx',
 './data/kot_data/MAY 2019 Time spent kot traking SWIGGY.xlsx',
 './data/kot_data/JAN 2019 ITEM SPENT KOT TRAKING SWIGGY  .xlsx',
 './data/kot_data/JULY 2019 ITEM SPENT KOT TRAKING SWIGGY.xlsx',
 './data/kot_data/JULY 2019 ITEM SPENT KOT TRAKING BAKERY .xlsx',
 './data/kot_data/MARCH 2019 ITEM SPENT KOT TRAKING ZOMATO.xlsx',
 './data/kot_data/JULY 2019 ITEM SPENT KOT TRAKING TIMBER .xlsx',
 './data/kot_data/SEP 2019 TIEM SPENT KOT TRAKING HD.xlsx',
 './data/kot_data/MARCH 2019 ITEM SPENT KOT TRAKING SHISH .xlsx',
 './data/kot_data/JAN 2019 ITEM SPENT KOT TRAKING THE ESTATE  .xlsx',
 './data/kot_data/MAY 2019 Time spent kot traking HD.xlsx',
 './data/kot_data/JAN 2019 ITEM SPENT KOT TRAKING TIMBER  .xlsx',
 './data/kot_data/SEP 2019 TIEM SPENT KOT TRAKING ZOMATO.xlsx',
 './data/kot_data/SEP 201

In [79]:
if os.path.exists("./clean_data/full_csv_kot.csv"):
    os.remove("./clean_data/full_csv_kot.csv")
    print("full_csv_kot Removed!")
    
with open('./clean_data/{}.csv'.format('full_csv_kot'), mode='w+') as f:
    f.write(','.join(list_cols.keys()))
    f.write('\n')
    
for x in list_of_files:
    try:
        workbook = xlrd.open_workbook(x)
        sheet = workbook.sheet_by_index(0)
        prepareCsvData(sheet, x)
    except Exception as e:
        print('file {} has following issues : \n {}'.format(x,e))

File is completed at 4466 and classification col 5 sheet rows : 4466
Completed file ./data/kot_data/MARCH 2019 ITEM SPENT KOT TRAKING TIMBER .xlsx with data written >>> 3954 after cursor at 4466
File is completed at 3634 and classification col 5 sheet rows : 3634
Completed file ./data/kot_data/April 2019 Time spent kot Traking TIMBER .xlsx with data written >>> 3203 after cursor at 3634
File is completed at 71 and classification col 5 sheet rows : 71
Completed file ./data/kot_data/MARCH 2019 ITEM SPENT KOT TRAKING HD.xlsx with data written >>> 54 after cursor at 71
File is completed at 336 and classification col 5 sheet rows : 336
Completed file ./data/kot_data/MAY 2019 Time spent kot traking SWIGGY.xlsx with data written >>> 253 after cursor at 336
File is completed at 369 and classification col 5 sheet rows : 369
Completed file ./data/kot_data/JAN 2019 ITEM SPENT KOT TRAKING SWIGGY  .xlsx with data written >>> 262 after cursor at 369
File is completed at 409 and classification col 5 

In [ ]:
'2190'.strip()